# Fine-Tuning Llama-3.2-3B with PEFT LoRA on a Single GPU

This notebook demonstrates how to efficiently fine-tune the Llama-3.2-3B model using Parameter-Efficient Fine-Tuning (PEFT) with LoRA adapters and 4-bit quantization via bitsandbytes.

Rather than updating all model parameters, LoRA (Low-Rank Adaptation) enables us to train small adapter modules that can be injected into the base model. This approach dramatically reduces memory requirements and training time. Once trained, these adapters can be easily shared via the Hugging Face Hub.

This technique works with any model that supports the `device_map` parameter through the accelerate library.

## Control Flags Reference

This notebook uses boolean flags to skip unnecessary steps on subsequent runs.

**First run (training):**
- Cell 5: `install_packages = True`
- Cell 14: `load_dataset_flag = True`
- Cell 15: `do_login = True`
- Cell 37: `install_trl = True`
- Cell 40: `run_training = True, resume_from_last = False`

**Resume training:**
- Cell 40: `run_training = True, resume_from_last = True`
- All other flags: `False`

**Export only (skip training):**
- Cell 40: `run_training = False`
- Cell 44: `save_merged_model = True` (for VLLM)
- Cell 46: `install_llamacpp = True` (first time)
- Cell 47: `convert_to_gguf = True` (for GGUF)
- All other flags: `False`

## Step 0 - Setup Helper Functions

We'll configure our notebook environment with two utility functions:
1. Enable automatic text wrapping in outputs to improve readability
2. Create an inference wrapper to query the model and retrieve generated responses

In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))

get_ipython().events.register('pre_run_cell', set_css)


Below is a helper function that will handle model inference by processing user queries and returning the model's generated response

## Step 1 - Install Required Libraries

Begin by installing the necessary dependencies from their source repositories to access the latest features for model quantization and fine-tuning.

In [ ]:
install_packages = True  # Set to True on first run only

if install_packages:
    !pip install -q -U bitsandbytes
    !pip install -q -U transformers  # Changed to install from PyPI
    !pip install -q -U git+https://github.com/huggingface/peft.git
    !pip install -q -U git+https://github.com/huggingface/accelerate.git
    !pip install -q datasets
else:
    print("✓ Skipping package installation (already installed)")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 47.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## Step 2 - Initialize the Model

We'll configure QLoRA (Quantized LoRA) settings to load the model in 4-bit precision, significantly reducing memory consumption while maintaining performance.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

Next, we define our model identifier and load both the model and tokenizer using the quantization settings configured above.

In [ ]:
model_id = "meta-llama/Llama-3.2-3B"

# model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto", token=hf_token)
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True, token=hf_token)

config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

Test the base model before fine-tuning. The pre-trained model may not follow instructions effectively without additional training.

In [ ]:
# result = get_completion(query="Will capital gains affect my tax bracket?", model=model, tokenizer=tokenizer)
# print(result)

## Step 3 - Prepare Training Data

Let's load the FineTome-100k dataset, a high-quality instruction dataset for fine-tuning. We'll use a subset of the data for this demo.

In [ ]:
load_dataset_flag = True  # Set to True only if you need to train

if load_dataset_flag:
    from datasets import load_dataset

    # Load the FineTome-100k dataset
    data = load_dataset("mlabonne/FineTome-100k", split="train")

    # Take a subset for faster training (you can adjust this)
    data = data.select(range(min(1000, len(data))))

    # Display first few examples
    df = data.to_pandas()
    print(df.head(10))
else:
    print("✓ Skipping dataset loading (not needed for exports)")

README.md:   0%|          | 0.00/982 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/117M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

                                       conversations  \
0  [{'from': 'human', 'value': 'Explain what bool...   
1  [{'from': 'human', 'value': 'Explain how recur...   
2  [{'from': 'human', 'value': 'Explain what bool...   
3  [{'from': 'human', 'value': 'Explain the conce...   
4  [{'from': 'human', 'value': 'Print the reverse...   
5  [{'from': 'human', 'value': 'How do astronomer...   
6  [{'from': 'human', 'value': 'Explain what is m...   
7  [{'from': 'human', 'value': 'Compare two strin...   
8  [{'from': 'human', 'value': 'Explain how a ter...   
9  [{'from': 'human', 'value': 'Develop a lesson ...   

                     source     score  
0  infini-instruct-top-500k  5.212621  
1  infini-instruct-top-500k  5.157649  
2  infini-instruct-top-500k  5.147540  
3  infini-instruct-top-500k  5.053656  
4  infini-instruct-top-500k  5.045648  
5    WebInstructSub_axolotl  5.025244  
6  infini-instruct-top-500k  5.022963  
7  infini-instruct-top-500k  5.007371  
8  infini-instruct-top-

In [ ]:
do_login = True  # Set to True once per session

if do_login:
    from huggingface_hub import login
    login(token=hf_token)
else:
    print("✓ Skipping HF login (already authenticated)")

In [ ]:
# Dataset is already loaded in the previous cell
# Display the dataset info
print(data)
print("\nDataset columns:", data.column_names)

Dataset({
    features: ['conversations', 'source', 'score'],
    num_rows: 1000
})

Dataset columns: ['conversations', 'source', 'score']


Instruction Finetuning - Prepare the dataset:
1. Format the conversations into a single text field
2. Shuffle the dataset
3. Tokenize the dataset

Our dataset needs to be properly tokenized so the model can process it during training.

In [ ]:
# Format conversations into text and tokenize
def format_conversation(example):
    # FineTome dataset has 'conversations' field with list of messages
    if 'conversations' in example:
        messages = example['conversations']
        text = ""
        for msg in messages:
            role = msg.get('from', msg.get('role', ''))
            content = msg.get('value', msg.get('content', ''))
            text += f"{role}: {content}\n"
        example['text'] = text

    # If dataset already has 'text', leave as is
    return example

# 1) Format conversation into plain text field
data = data.map(format_conversation)

# 2) Shuffle
data = data.shuffle(seed=1234)

# 3) Correct tokenization (the part that was wrong)
def tokenize_fn(samples):
    texts = samples["text"]  # now we are sure it exists
    return tokenizer(
        texts,
        truncation=True,
        max_length=512,
        padding=False  # important for Colab RAM
    )

data = data.map(
    tokenize_fn,
    batched=True,
    remove_columns=data.column_names
)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Create train/test split with 80% for training and 20% for evaluation

In [ ]:
data = data.train_test_split(test_size=0.2)
train_data = data["train"]
test_data = data["test"]


In [ ]:
print(test_data)

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 200
})


## Step 4 - Configure LoRA Adapters

Now we'll apply the PEFT library to configure LoRA (Low-Rank Adaptation) for efficient fine-tuning. We use `prepare_model_for_kbit_training` to set up the model for training with quantized weights.

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm

Identify all linear layers in the model architecture for LoRA adaptation.

According to the QLoRA research: "Our experiments show that applying LoRA to all linear layers in the transformer blocks is essential for achieving performance comparable to full fine-tuning. The total number of LoRA adapters is the most important hyperparameter."

In [ ]:
import bitsandbytes as bnb
def find_all_linear_names(model):
  cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
  lora_module_names = set()
  for name, module in model.named_modules():
    if isinstance(module, cls):
      names = name.split('.')
      lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names: # needed for 16-bit
      lora_module_names.remove('lm_head')
  return list(lora_module_names)

In [ ]:
modules = find_all_linear_names(model)
print(modules)

['v_proj', 'k_proj', 'o_proj', 'gate_proj', 'up_proj', 'q_proj', 'down_proj']


In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=modules,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)

In [ ]:
trainable, total = model.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")


Trainable: 24313856 | total: 3237063680 | Percentage: 0.7511%


## Step 5 - Execute Training

In [ ]:
# Authentication already handled in previous cells

Configure training hyperparameters and checkpoint saving:
* Checkpoints are saved every 25 steps in PyTorch/safetensors format
* For demonstration purposes, we're running 100 training steps. Adjust this based on your needs and available resources.
* Optional: Mount Google Drive below to persist checkpoints

In [ ]:
# from datasets import load_dataset
# data = load_dataset("ronal999/finance-alpaca-demo", split='train')
# data = data.train_test_split(test_size=0.1)
# train_data = data["train"]
# test_data = data["test"]

In [ ]:
# import transformers

# tokenizer.pad_token = tokenizer.eos_token


# trainer = transformers.Trainer(
#     model=model,
#     train_dataset=train_data,
#     eval_dataset=test_data,
#     args=transformers.TrainingArguments(
#         per_device_train_batch_size=1,
#         gradient_accumulation_steps=4,
#         warmup_steps=0.03,
#         max_steps=100,
#         learning_rate=2e-4,
#         fp16=True,
#         logging_steps=1,
#         output_dir="outputs_mistral_b_finance_finetuned_test",
#         optim="paged_adamw_8bit",
#         save_strategy="epoch",
#     ),
#     data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
# )


In [ ]:
install_trl = True  # Set to True on first run only

if install_trl:
    !pip install -q trl
else:
    print("✓ Skipping TRL installation (already installed)")

# Optional: Mount Google Drive to save checkpoints persistently
# Uncomment the lines below if you want to save to Google Drive
# from google.colab import drive
# drive.mount('/content/drive')
# Then change output_dir in cell below to: "/content/drive/MyDrive/outputs_llama_1b_finetuned"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 8.9 MB/s eta 0:00:00


In [ ]:
import transformers
from trl import SFTTrainer, SFTConfig

tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

torch.cuda.empty_cache()

model.gradient_checkpointing_enable()
model.enable_input_require_grads()
torch.cuda.empty_cache()

from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    data_collator=data_collator,     # <-- corretto
    peft_config=lora_config,
    args=SFTConfig(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=8,
        warmup_steps=20,
        max_steps=150,
        learning_rate=2e-4,
        logging_steps=1,
        eval_strategy="no",
        output_dir="outputs_llama_3b",
        optim="paged_adamw_8bit",
        save_strategy="steps",
        save_steps=50,
        save_total_limit=3,
        save_safetensors=True,
        save_only_model=False,
        report_to="none",
    )
)

/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:397: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Begin the training process

In [ ]:
# Training with auto-resume support
run_training = True  # Set to True to start/resume training
resume_from_last = False  # Set to True to resume from last checkpoint

if run_training:
    model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

    # Auto-detect last checkpoint if resume_from_last is True
    checkpoint_path = None
    if resume_from_last:
        import os
        output_dir = "outputs_llama_3b_finetuned"

        if os.path.exists(output_dir):
            checkpoints = [d for d in os.listdir(output_dir)
                          if d.startswith("checkpoint-")]
            if checkpoints:
                # Sort by step number and get the latest
                latest_checkpoint = sorted(checkpoints,
                                         key=lambda x: int(x.split("-")[1]))[-1]
                checkpoint_path = f"{output_dir}/{latest_checkpoint}"
                print(f"📂 Resuming from: {checkpoint_path}")
            else:
                print("⚠ No checkpoints found, starting from scratch")
        else:
            print("⚠ Output directory doesn't exist, starting from scratch")

    # Train (will resume if checkpoint_path is set)
    trainer.train(resume_from_checkpoint=checkpoint_path)

    # Save the final model locally in PyTorch format
    model.save_pretrained("final_llama_3b_model")
    tokenizer.save_pretrained("final_llama_3b_model")
    print("\n✓ Training complete! Final model saved to: final_llama_3b_model/")
    print(f"✓ Checkpoints saved in: outputs_llama_3b_finetuned/")
else:
    print("Set run_training = True to start/resume training")
    print("\nTo resume from a checkpoint:")
    print("  1. Set run_training = True")
    print("  2. Set resume_from_last = True")
    print("  3. Run this cell")

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': None}.
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
1,1.275900
2,1.382400
3,1.146700
4,1.186200
5,1.084900
6,1.089100
7,1.286900
8,0.999200
9,1.138600
10,1.306700


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
1,1.275900
2,1.382400
3,1.146700
4,1.186200
5,1.084900
6,1.089100
7,1.286900
8,0.999200
9,1.138600
10,1.306700


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)



✓ Training complete! Final model saved to: final_llama_3b_model/
✓ Checkpoints saved in: outputs_llama_3b_finetuned/


## Load a Checkpoint (Optional)

If you need to load a specific checkpoint to resume training or for inference:

```python
# Load from a specific checkpoint
from peft import PeftModel
checkpoint_path = "outputs_llama_3b_finetuned/checkpoint-75"  # Change to your checkpoint number

# Or load the final saved model
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
model = PeftModel.from_pretrained(model, "final_llama_3b_model")
```

Upload the fine-tuned model and adapters to the Hugging Face Hub

In [ ]:
# Push the final model to HuggingFace Hub
model.push_to_hub("Zedel17/fine_tuned_llama_3b")
tokenizer.push_to_hub("Zedel17/fine_tuned_llama_3b")

print("\n✓ Model successfully uploaded to: https://huggingface.co/Zedel17/fine_tuned_llama_3b")

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...adapter_model.safetensors:   0%|          | 45.8kB / 97.3MB            

README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...mp5ef_ad7e/tokenizer.json: 100%|##########| 17.2MB / 17.2MB            


✓ Model successfully uploaded to: https://huggingface.co/Zedel17/fine_tuned_llama_3b


## Step 6 - Saving to Float16 for VLLM

VLLM (Very Large Language Model) inference engine works best with merged models in float16 format. Here we'll merge the LoRA adapters back into the base model and save it in a format optimized for VLLM deployment.

This process:
1. Loads the base model in full precision
2. Merges the trained LoRA adapters
3. Converts to float16 to reduce model size
4. Saves in a format ready for VLLM inference

In [ ]:
# Merge LoRA adapters and save to float16 for VLLM
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# Set to True to perform the merge and save
save_merged_model = True

if save_merged_model:
    print("Loading base model in float16...")
    # Load base model in float16 (no quantization for merging)
    base_model = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch.float16,
        device_map="auto",
        token=hf_token
    )

    print("Loading and merging LoRA adapters...")
    # Load the PEFT model (LoRA adapters)
    model_with_adapters = PeftModel.from_pretrained(base_model, "final_llama_3b_model")

    # Merge adapters into base model
    merged_model = model_with_adapters.merge_and_unload()

    print("Saving merged model in float16 format...")
    # Save the merged model
    output_dir = "llama_3b_merged_float16"
    merged_model.save_pretrained(
        output_dir,
        safe_serialization=True,  # Save in safetensors format
        max_shard_size="5GB"      # Shard if model is large
    )

    # Save tokenizer
    tokenizer_merged = AutoTokenizer.from_pretrained(model_id, token=hf_token)
    tokenizer_merged.save_pretrained(output_dir)

    print(f"\n✓ Merged model saved to: {output_dir}/")
    print("✓ This model is ready for VLLM deployment!")

    # Optional: Push to Hugging Face Hub
    push_to_hub = True
    if push_to_hub:
        merged_model.push_to_hub("Zedel17/llama_3b_merged_float16")
        tokenizer_merged.push_to_hub("Zedel17/llama_3b_merged_float16")
        print("✓ Merged model uploaded to Hugging Face Hub!")
else:
    print("Set save_merged_model = True to merge and save the model for VLLM")

Loading base model in float16...


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading and merging LoRA adapters...
Saving merged model in float16 format...

✓ Merged model saved to: llama_3b_merged_float16/
✓ This model is ready for VLLM deployment!


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

...

README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...ed_float16/tokenizer.json: 100%|##########| 17.2MB / 17.2MB            

✓ Merged model uploaded to Hugging Face Hub!


## Step 7 - GGUF Conversion for llama.cpp

GGUF (GPT-Generated Unified Format) is the format used by llama.cpp for efficient CPU and GPU inference. This section will convert your fine-tuned model to GGUF format with various quantization options.

Supported quantization methods:
* `f16` - Full float16 precision (largest file, best quality)
* `q8_0` - 8-bit quantization (good balance, fast conversion)
* `q4_k_m` - 4-bit quantization, medium variant (recommended for most use cases)
* `q5_k_m` - 5-bit quantization, medium variant (good quality/size trade-off)
* `q2_k` - 2-bit quantization (smallest size, lower quality)

The conversion process:
1. First ensure you have the merged float16 model (from Step 6)
2. Clone and set up llama.cpp
3. Run the conversion script with your chosen quantization method

In [23]:
# Install dependencies and clone llama.cpp
install_llamacpp = True

if install_llamacpp:
    print("Installing required packages...")
    !pip install -q -U gguf

    print("\nCloning llama.cpp repository...")
    !git clone https://github.com/ggerganov/llama.cpp.git

    print("\nInstalling llama.cpp Python requirements...")
    !pip install -q -r llama.cpp/requirements.txt

    print("\n✓ llama.cpp setup complete!")
else:
    print("Set install_llamacpp = True to install llama.cpp tools")

Installing required packages...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 3.8 MB/s eta 0:00:00

Cloning llama.cpp repository...
Cloning into 'llama.cpp'...
remote: Enumerating objects: 70476, done.
remote: Counting objects: 100% (140/140), done.
remote: Compressing objects: 100% (106/106), done.
remote: Total 70476 (delta 74), reused 34 (delta 34), pack-reused 70336 (from 3)
Receiving objects: 100% (70476/70476), 220.38 MiB | 24.17 MiB/s, done.
Resolving deltas: 100% (50828/50828), done.

Installing llama.cpp Python requirements...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 51.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 32.1 MB/s eta 0:00:00
   ━━━

In [ ]:
# Convert merged model to GGUF format
convert_to_gguf = True

if convert_to_gguf:
    import os
    import shutil

    merged_model_path = "llama_3b_merged_float16"
    output_base_name = "llama_3b_finetuned"

    print("Ensuring tokenizer files are complete...")
    tokenizer_config_src = f"{merged_model_path}/tokenizer_config.json"
    if os.path.exists(tokenizer_config_src):
        print("✓ Tokenizer files found")
    else:
        print("⚠ Tokenizer missing - this shouldn't happen")

    # Build llama-quantize if needed
    if not os.path.exists("llama.cpp/build/bin/llama-quantize"):
        print("\nBuilding llama-quantize binary...")
        %cd llama.cpp
        !cmake -B build
        !cmake --build build --config Release -j4
        %cd ..
        print("✓ Built!\n")

    print("Step 1: Converting to f16 GGUF format...")
    !python llama.cpp/convert_hf_to_gguf.py {merged_model_path} \
        --outfile {output_base_name}-f16.gguf \
        --outtype f16

    if not os.path.exists(f"{output_base_name}-f16.gguf"):
        print("\nf16 conversion failed")
        print("\nTrying alternative: q8_0 conversion...")
        !python llama.cpp/convert_hf_to_gguf.py {merged_model_path} \
            --outfile {output_base_name}-Q8_0.gguf \
            --outtype q8_0

        if os.path.exists(f"{output_base_name}-Q8_0.gguf"):
            print("\n✓ Q8_0 conversion succeeded")
            base_file = f"{output_base_name}-Q8_0.gguf"
        else:
            print("\nAll conversions failed.")
            base_file = None
    else:
        size_mb = os.path.getsize(f"{output_base_name}-f16.gguf") / (1024 * 1024)
        print(f"✓ Created: {output_base_name}-f16.gguf ({size_mb:.2f} MB)")
        base_file = f"{output_base_name}-f16.gguf"

    if base_file:
        print("\nStep 2: Quantizing to Q4_K_M...")
        output_file = f"{output_base_name}-Q4_K_M.gguf"
        !./llama.cpp/build/bin/llama-quantize {base_file} {output_file} q4_k_m

        if os.path.exists(output_file):
            size_mb = os.path.getsize(output_file) / (1024 * 1024)
            print(f"✓ Created: {output_file} ({size_mb:.2f} MB)")
            print("\nGGUF Conversion Complete!")
        else:
            print("Quantization failed")

else:
    print("Set convert_to_gguf = True to convert")

Ensuring tokenizer files are complete...
✓ Tokenizer files found

Building llama-quantize binary...
/content/llama.cpp
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
CMAKE_BUILD_TYPE=Release
-- Found Git: /usr/bin/git (found version "2.34.1")
-- The ASM compiler identification is GNU
-- Found assembler: /usr/bin/cc
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Success
-- Found Threads: TRUE
-- Warning: ccache not found - consider installing it for faster compilation or disable this warning with 

In [ ]:
# Optional: Upload GGUF files to Hugging Face Hub
upload_gguf_to_hub = True

if upload_gguf_to_hub:
    from huggingface_hub import HfApi, create_repo

    api = HfApi()
    repo_id = "Zedel17/llama_3b_gguf"  # Change to your username

    # Create repository if it doesn't exist
    try:
        create_repo(repo_id, repo_type="model", exist_ok=True)
        print(f"Repository created/verified: {repo_id}")
    except Exception as e:
        print(f"Repository setup: {e}")

    # Upload GGUF files
    gguf_files = [
        "llama_1b_finetuned-f16.gguf",
        "llama_1b_finetuned-Q8_0.gguf",
        "llama_1b_finetuned-Q4_K_M.gguf",
        "llama_1b_finetuned-Q5_K_M.gguf",
    ]

    print("\nUploading GGUF files...")
    for gguf_file in gguf_files:
        if os.path.exists(gguf_file):
            print(f"  Uploading {gguf_file}...")
            api.upload_file(
                path_or_fileobj=gguf_file,
                path_in_repo=gguf_file,
                repo_id=repo_id,
                repo_type="model",
            )
            print(f"  ✓ Uploaded {gguf_file}")
        else:
            print(f"  ⚠ File not found: {gguf_file}")

    print(f"\n✓ GGUF files uploaded to: https://huggingface.co/{repo_id}")
else:
    print("Set upload_gguf_to_hub = True to upload GGUF files to Hugging Face Hub")

Repository created/verified: Zedel17/llama_1b_gguf

Uploading GGUF files...
  Uploading llama_1b_finetuned-f16.gguf...


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  llama_1b_finetuned-f16.gguf :   1%|1         | 34.0MB / 2.48GB            

  ✓ Uploaded llama_1b_finetuned-f16.gguf
  ⚠ File not found: llama_1b_finetuned-Q8_0.gguf
  Uploading llama_1b_finetuned-Q4_K_M.gguf...


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ..._1b_finetuned-Q4_K_M.gguf:   1%|          | 7.55MB /  808MB            

  ✓ Uploaded llama_1b_finetuned-Q4_K_M.gguf
  ⚠ File not found: llama_1b_finetuned-Q5_K_M.gguf

✓ GGUF files uploaded to: https://huggingface.co/Zedel17/llama_1b_gguf
